In [1]:
import pandas as pd
import numpy as np
import warnings
import re
from tqdm import tqdm_notebook

warnings.filterwarnings('ignore')
pd.options.display.max_columns = 50

In [2]:
brands_dict = {
    'audi': 'ASgBAgICAUTgtg3elyg',
    'bmw': 'ASgBAgICAUTgtg3klyg',
    'changan': 'ASgBAgICAUTgtg3wlyg',
    'chery': 'ASgBAgICAUTgtg30lyg',
    'chevrolet': 'ASgBAgICAUTgtg32lyg',
    'citroen': 'ASgBAgICAUTgtg36lyg',
    'exeed': 'ASgBAgICAUTgtg24ldEB',
    'ford': 'ASgBAgICAUTgtg2cmCg',
    'geely': 'ASgBAgICAUTgtg2gmCg',
    'haval': 'ASgBAgICAUTgtg2umCg',
    'honda': 'ASgBAgICAUTgtg2ymCg',
    'hyundai': 'ASgBAgICAUTgtg2imzE',
    'kia': 'ASgBAgICAUTgtg3KmCg',
    'land_rover': 'ASgBAgICAUTgtg3QmCg',
    'lexus': 'ASgBAgICAUTgtg3WmCg',
    'mazda': 'ASgBAgICAUTgtg3mmCg',
    'mercedes-benz': 'ASgBAgICAUTgtg3omCg',
    'mitsubishi': 'ASgBAgICAUTgtg3ymCg',
    'nissan': 'ASgBAgICAUTgtg36mCg',
    'opel': 'ASgBAgICAUTgtg3',
    'peugeot': 'ASgBAgICAUTgtg2AmSg',
    'renault': 'ASgBAgICAUTgtg2MmSg',
    'skoda': 'ASgBAgICAUTgtg2emSg',
    'toyota': 'ASgBAgICAUTgtg20mSg',
    'volkswagen': 'ASgBAgICAUTgtg24mSg',
    'volvo': 'ASgBAgICAUTgtg26mSg',
    'vaz_lada': 'ASgBAgICAUTgtg3GmSg',
    'gaz': 'ASgBAgICAUTgtg3KmSg'
}

# Подготовка и формирование единого датасета.

Соберём в единый датасет наши сырые куски данных.

In [3]:
df = pd.DataFrame()
for brand in tqdm_notebook(list(brands_dict.keys())):
    for page in range(1, 100):
        try:
            df = pd.concat(
                [df, pd.read_csv(f'./parsed_data/data_{brand}_{page}.csv')],
                ignore_index=True)
        except:
            continue

  0%|          | 0/28 [00:00<?, ?it/s]

In [4]:
df.shape

(101110, 201)

Сразу выкинем все дублирующиеся объекты и некорректные строки.

In [5]:
df = df.drop_duplicates()

In [6]:
df.shape

(37573, 201)

In [7]:
df[['Поколение', 'Название авто']][df['Поколение'] == 'Поколение']

,Поколение,Название авто
815,Поколение,Название авто


In [8]:
df = df.drop(815, axis=0)

In [9]:
df = df.reset_index().drop('index', axis=1)

In [10]:
df[:5]

,Название авто,Рейтинг,Цена,Расположение,Год выпуска,Поколение,Пробег,История пробега,Владельцев по ПТС,Состояние,Модификация,Объём двигателя,Тип двигателя,Коробка передач,Привод,Комплектация,Тип кузова,Цвет,Руль,VIN или номер кузова,Подушки безопасности,Коленные,Боковые передние,Электрический,Управление климатом,...,Мультимедиа и навигацияCD/DVD/Blu-rayMP3USBBluetooth,Управление на рулеGPS-навигатор,Мультимедиа и навигацияGPS-навигатор,РадиоTV,Управление на рулеUSBBluetoothGPS-навигатор,Мультимедиа и навигацияCD/DVD/Blu-rayMP3USBAUXBluetoothGPS-навигатор,Мультимедиа и навигацияCD/DVD/Blu-rayGPS-навигатор,Мультимедиа и навигацияCD/DVD/Blu-rayAUXGPS-навигатор,Мультимедиа и навигацияTV,Мультимедиа и навигацияTVUSBAUXBluetooth,ВидеоAUXBluetoothGPS-навигатор,Мультимедиа и навигацияCD/DVD/Blu-rayMP3USBBluetoothGPS-навигатор,ВидеоAUXBluetooth,Мультимедиа и навигацияCD/DVD/Blu-rayBluetooth,ВидеоBluetooth,Управление на рулеAUXBluetoothGPS-навигатор,Мультимедиа и навигацияCD/DVD/Blu-rayUSBAUXBluetoothGPS-навигатор,"Шины и диски25""",РадиоAUXBluetooth,Мультимедиа и навигацияUSBAUXBluetoothGPS-навигатор,Мультимедиа и навигацияMP3BluetoothGPS-навигатор,ВидеоAUX,Мультимедиа и навигацияMP3AUXBluetooth,Мультимедиа и навигацияMP3USBGPS-навигатор,Мультимедиа и навигацияCD/DVD/Blu-rayUSBGPS-навигатор
0,"Audi Q3 2.0 AMT, 2013, 166 000 км","4,8",1745000,"Самарская обл., Самара, Московское ш., 262А",2013,I (2011—2014),166 000 км,11 записей в отчёте Автотеки,3,Не битый,2.0 TFSI quattro S tronic (170 л.с.),2 л,Бензин,Робот,Полный,Base,Внедорожник 5-дверный,Чёрный,Левый,WAUZ*************,Фронтальные,Шторки,Усилитель руля,Управление климатом,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Audi Q7 3.0 AT, 2021, 29 226 км","5,0",8785500,"Москва, Волоколамское ш., 73",2021,4M рестайлинг (2019—2024),29 226 км,2 записи в отчёте Автотеки,1,Не битый,45 TDI 3.0 quattro Tiptronic (249 л.с.),3 л,Дизель,Автомат,Полный,Sport,Внедорожник 5-дверный,Чёрный,Левый,XW8Z*************,NaN,NaN,NaN,NaN,Климат-контроль многозонный,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Audi Q8 3.0 AT, 2022, 100 км","5,0",11200000,"Москва, ул. Верхние Поля, 2",2022,I (2018—2023),100 км,3 записи в отчёте Автотеки,1,Не битый,55 TFSI 3.0 quattro Tiptronic (340 л.с.),3 л,Бензин,Автомат,Полный,Advance,Внедорожник 5-дверный,Чёрный,Левый,WAUZ*************,NaN,NaN,NaN,NaN,Климат-контроль многозонный,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Audi Q7 3.0 AT, 2013, 151 488 км","4,8",2099000,"Санкт-Петербург, пр-т Маршала Жукова, 78",2013,4L рестайлинг (2009—2015),151 488 км,15 записей в отчёте Автотеки,2,Не битый,3.0 TDI quattro Tiptronic (245 л.с.),3 л,Дизель,Автомат,Полный,NaN,Внедорожник 5-дверный,Белый,Левый,XW8Z*************,NaN,NaN,NaN,NaN,Климат-контроль многозонный,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Audi Q8 3.0 AT, 2022, 3 261 км","5,0",10400000,"Москва, Бережковская наб., 38с2",2022,I (2018—2023),3 261 км,NaN,1,Не битый,50 TDI 3.0 quattro Tiptronic (286 л.с.),3 л,Дизель,Автомат,Полный,Base,Внедорожник 5-дверный,Красный,Левый,WAUZ*************,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Первичный обзор признаков и проверка их на заполненность.

201 признак у каждого объекта. Посмотрим на них поближе.

In [11]:
df.columns[50:100]

Index(['Боковые задние', 'Гидравлический', 'Помощь при вождении',
       'Цвет салона', 'Обогрев', 'Шины и диски17"', 'Парктроник передний',
       'Бортовой компьютер', 'Климат-контроль многозонный', 'Зеркал', 'Фары',
       'Память настроек', 'Шины и диски19"', 'Электропривод',
       'Блок. дифференциала', 'Электростеклоподъемники',
       'Мультимедиа и навигацияBluetooth', 'Складывания зеркал',
       'Заднего стекла', 'Экстренное торможение', 'Атермальное остекление',
       'Данные о ', 'О', 'Обслуживался у дилера', 'Противотуманные',
       'Кондиционер', 'Адаптивное освещение', 'Зимние шины в комплекте',
       'Шины и диски', 'Мультимедиа и навигацияMP3USBAUX', 'Круиз-контроль',
       'Датчик света', 'Шины и диски20"', 'Рулевой колонки', 'Кожа',
       'Аудиосистема4 колонки', 'Цветной', 'Передние и задние',
       'Парктроник задний', 'Мультимедиа и навигацияMP3',
       'ВидеоUSBAUXBluetooth', 'Сабвуфер', 'Задних сидений',
       'Аудиосистема8+ колонок', 'Ксеноновые', 'Ши

In [12]:
df.columns[:100]

Index(['Название авто', 'Рейтинг', 'Цена', 'Расположение', 'Год выпуска',
       'Поколение', 'Пробег', 'История пробега', 'Владельцев по ПТС',
       'Состояние', 'Модификация', 'Объём двигателя', 'Тип двигателя',
       'Коробка передач', 'Привод', 'Комплектация', 'Тип кузова', 'Цвет',
       'Руль', 'VIN или номер кузова', 'Подушки безопасности', 'Коленные',
       'Боковые передние', 'Электрический', 'Управление климатом', 'Салон',
       'Кожаный руль', 'Центральный замок', 'ПТС', 'Обмен', 'Усилитель руля',
       'Управление на руле', 'Противоугонная система', 'Сигнализация',
       'Фронтальные', 'Мультимедиа и навигацияCD/DVD/Blu-rayMP3',
       'Управление на рулеUSBAUXBluetoothGPS-навигатор', 'Комбинированный',
       'Люк', 'Активная безопасность', 'Курсовая устойчивость', 'Шторки',
       'Передних сидений', 'Иммобилайзер', 'Антиблокировка тормозов',
       'Спутник', 'Электрогидравлический', 'Антипробуксовка',
       'Распред. тормозных усилий', 'Ткань', 'Боковые задние',


In [13]:
df.columns[100:]

Index(['Тёмный', 'Датчик дождя', 'Мультимедиа и навигацияAUX', 'Руля',
       'Мультимедиа и навигацияUSBAUX', 'Только передние',
       'Аудиосистема2 колонки', 'Видео',
       'Мультимедиа и навигацияCD/DVD/Blu-rayMP3USBAUXBluetooth',
       'Омыватели фар',
       ...
       'Управление на рулеAUXBluetoothGPS-навигатор',
       'Мультимедиа и навигацияCD/DVD/Blu-rayUSBAUXBluetoothGPS-навигатор',
       'Шины и диски25"', 'РадиоAUXBluetooth',
       'Мультимедиа и навигацияUSBAUXBluetoothGPS-навигатор',
       'Мультимедиа и навигацияMP3BluetoothGPS-навигатор', 'ВидеоAUX',
       'Мультимедиа и навигацияMP3AUXBluetooth',
       'Мультимедиа и навигацияMP3USBGPS-навигатор',
       'Мультимедиа и навигацияCD/DVD/Blu-rayUSBGPS-навигатор'],
      dtype='object', length=101)

Признаки, заполненные менее чем на 50% для нас не представляют особой ценности. Их мы удалим.

In [14]:
filter_mask = (df.isna().sum() / df.shape[0] * 100) > 50

In [15]:
df = df[df.columns[~filter_mask]]

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37572 entries, 0 to 37571
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Название авто         37572 non-null  object
 1   Рейтинг               37572 non-null  object
 2   Цена                  37572 non-null  object
 3   Расположение          37572 non-null  object
 4   Год выпуска           37568 non-null  object
 5   Поколение             37568 non-null  object
 6   Пробег                33530 non-null  object
 7   История пробега       31357 non-null  object
 8   Владельцев по ПТС     33530 non-null  object
 9   Состояние             37568 non-null  object
 10  Модификация           37568 non-null  object
 11  Объём двигателя       37442 non-null  object
 12  Тип двигателя         37568 non-null  object
 13  Коробка передач       37568 non-null  object
 14  Привод                37568 non-null  object
 15  Комплектация          30290 non-null

# Формирование новых и первичное преобразование признаков.

Из тех признаков что есть, мы получим новые: например из названия авто можно выделить бренд и модель. А также в некоторых случаях и модификацию. Также удалим лишние символы в данных.

## Модель, бренд.

In [17]:
def brand_name(auto_name: str) -> str:
    '''Функция для получения названия бренда из строки. 
       Принимает на вход строку и возвращает название бренда авто. '''
    splitted = auto_name.split(' ')
    return splitted[0]

In [18]:
def model_name(auto_name: str) -> str:
    '''Функция для получения модели авто из строки. 
       Принимает на вход строку и возвращает модель авто.'''
    try:
        modified_name = auto_name.replace(brand_name(auto_name) + ' ',
                                          '').replace('(',
                                                      '').replace(')',
                                                                  '').strip()
        splitted_name = modified_name.split(',')[0].strip()
        model_pattern = r'^(.*?)\s*\d+\.\d+\s*[A-Z]+'
        search_result = re.search(model_pattern, splitted_name)
        if search_result:
            return search_result.group(1)
        return splitted_name.replace(' AT', '')
    except Exception as err:
        print(auto_name + '//', splitted_name + '//', err)

In [19]:
df['Бренд авто'] = df['Название авто'].apply(brand_name)

In [20]:
df['Модель авто'] = df['Название авто'].apply(model_name)

In [21]:
df[:5]

,Название авто,Рейтинг,Цена,Расположение,Год выпуска,Поколение,Пробег,История пробега,Владельцев по ПТС,Состояние,Модификация,Объём двигателя,Тип двигателя,Коробка передач,Привод,Комплектация,Тип кузова,Цвет,Руль,VIN или номер кузова,Управление климатом,ПТС,Обмен,Бренд авто,Модель авто
0,"Audi Q3 2.0 AMT, 2013, 166 000 км","4,8",1745000,"Самарская обл., Самара, Московское ш., 262А",2013,I (2011—2014),166 000 км,11 записей в отчёте Автотеки,3,Не битый,2.0 TFSI quattro S tronic (170 л.с.),2 л,Бензин,Робот,Полный,Base,Внедорожник 5-дверный,Чёрный,Левый,WAUZ*************,NaN,NaN,NaN,Audi,Q3
1,"Audi Q7 3.0 AT, 2021, 29 226 км","5,0",8785500,"Москва, Волоколамское ш., 73",2021,4M рестайлинг (2019—2024),29 226 км,2 записи в отчёте Автотеки,1,Не битый,45 TDI 3.0 quattro Tiptronic (249 л.с.),3 л,Дизель,Автомат,Полный,Sport,Внедорожник 5-дверный,Чёрный,Левый,XW8Z*************,Климат-контроль многозонный,NaN,NaN,Audi,Q7
2,"Audi Q8 3.0 AT, 2022, 100 км","5,0",11200000,"Москва, ул. Верхние Поля, 2",2022,I (2018—2023),100 км,3 записи в отчёте Автотеки,1,Не битый,55 TFSI 3.0 quattro Tiptronic (340 л.с.),3 л,Бензин,Автомат,Полный,Advance,Внедорожник 5-дверный,Чёрный,Левый,WAUZ*************,Климат-контроль многозонный,Электронный,Не интересует,Audi,Q8
3,"Audi Q7 3.0 AT, 2013, 151 488 км","4,8",2099000,"Санкт-Петербург, пр-т Маршала Жукова, 78",2013,4L рестайлинг (2009—2015),151 488 км,15 записей в отчёте Автотеки,2,Не битый,3.0 TDI quattro Tiptronic (245 л.с.),3 л,Дизель,Автомат,Полный,NaN,Внедорожник 5-дверный,Белый,Левый,XW8Z*************,Климат-контроль многозонный,NaN,Возможен,Audi,Q7
4,"Audi Q8 3.0 AT, 2022, 3 261 км","5,0",10400000,"Москва, Бережковская наб., 38с2",2022,I (2018—2023),3 261 км,NaN,1,Не битый,50 TDI 3.0 quattro Tiptronic (286 л.с.),3 л,Дизель,Автомат,Полный,Base,Внедорожник 5-дверный,Красный,Левый,WAUZ*************,NaN,NaN,NaN,Audi,Q8


In [22]:
df['Бренд авто'].value_counts()

Бренд авто
Renault          1742
Chevrolet        1706
Hyundai          1602
Mercedes-Benz    1579
Honda            1539
Mazda            1518
Nissan           1507
BMW              1498
Mitsubishi       1481
Audi             1447
Lexus            1426
Toyota           1420
Land             1414
ГАЗ              1392
Kia              1377
Haval            1355
Skoda            1347
Volkswagen       1335
Ford             1334
ВАЗ              1295
Geely            1274
Chery            1239
Peugeot          1209
EXEED            1147
Citroen          1101
Changan          1001
Volvo             855
Opel              432
Name: count, dtype: int64

In [23]:
df.loc[df['Бренд авто']=='Land', 'Бренд авто'] = 'Land Rover'

In [24]:
df['Бренд авто'].value_counts()

Бренд авто
Renault          1742
Chevrolet        1706
Hyundai          1602
Mercedes-Benz    1579
Honda            1539
Mazda            1518
Nissan           1507
BMW              1498
Mitsubishi       1481
Audi             1447
Lexus            1426
Toyota           1420
Land Rover       1414
ГАЗ              1392
Kia              1377
Haval            1355
Skoda            1347
Volkswagen       1335
Ford             1334
ВАЗ              1295
Geely            1274
Chery            1239
Peugeot          1209
EXEED            1147
Citroen          1101
Changan          1001
Volvo             855
Opel              432
Name: count, dtype: int64

In [25]:
df.loc[df['Бренд авто'] == 'Land Rover',
       'Модель авто'] = df.loc[df['Бренд авто'] == 'Land Rover',
                               'Модель авто'].str.split().str[1:].str.join(' ')

In [26]:
df.loc[df['Бренд авто']=='Land Rover', 'Модель авто'][:5]

17620    Range Rover Sport
17621    Range Rover Sport
17622           Freelander
17623    Range Rover Sport
17624           Freelander
Name: Модель авто, dtype: object

## Пробег.

In [27]:
df = df.rename(columns={'Пробег': 'Пробег, км'})

In [28]:
def odometer_transformer(odom_num: str) -> int:
    '''Функция для получения стоимости в формате int из строки. 
       Принимает на вход строку, возвращает целое число. '''
    try:
        odom_num = int(re.sub(r'[^\d*]', '', odom_num))
        return odom_num
    except Exception as err:
        print(err, odom_num)

In [29]:
df['Пробег, км'] = df['Пробег, км'].apply(odometer_transformer)

expected string or bytes-like object nan
expected string or bytes-like object nan
expected string or bytes-like object nan
expected string or bytes-like object nan
expected string or bytes-like object nan
expected string or bytes-like object nan
expected string or bytes-like object nan
expected string or bytes-like object nan
expected string or bytes-like object nan
expected string or bytes-like object nan
expected string or bytes-like object nan
expected string or bytes-like object nan
expected string or bytes-like object nan
expected string or bytes-like object nan
expected string or bytes-like object nan
expected string or bytes-like object nan
expected string or bytes-like object nan
expected string or bytes-like object nan
expected string or bytes-like object nan
expected string or bytes-like object nan
expected string or bytes-like object nan
expected string or bytes-like object nan
expected string or bytes-like object nan
expected string or bytes-like object nan
expected string 

In [30]:
df['Пробег, км'][:5]

0    166000.0
1     29226.0
2       100.0
3    151488.0
4      3261.0
Name: Пробег, км, dtype: float64

## Рейтинг.

In [31]:
def rating_transformer(rating: str) -> float:
    '''Функция для получения рейтинга авто в формате числа с плавающей точкой.
       Принимает на вход строку, возвращает число с плавающей точкой.'''
    try:
        rating = rating.replace(',', '.')
        rating = float(rating)
        return rating
    except Exception as err:
        print(err, rating)

In [32]:
df['Рейтинг'] = df['Рейтинг'].apply(rating_transformer)

could not convert string to float: 'Отсутствует' Отсутствует
could not convert string to float: 'Отсутствует' Отсутствует
could not convert string to float: 'Отсутствует' Отсутствует
could not convert string to float: 'Отсутствует' Отсутствует
could not convert string to float: 'Отсутствует' Отсутствует
could not convert string to float: 'Отсутствует' Отсутствует
could not convert string to float: 'Отсутствует' Отсутствует
could not convert string to float: 'Отсутствует' Отсутствует
could not convert string to float: 'Отсутствует' Отсутствует
could not convert string to float: 'Отсутствует' Отсутствует
could not convert string to float: 'Отсутствует' Отсутствует
could not convert string to float: 'Отсутствует' Отсутствует
could not convert string to float: 'Отсутствует' Отсутствует
could not convert string to float: 'Отсутствует' Отсутствует
could not convert string to float: 'Отсутствует' Отсутствует
could not convert string to float: 'Отсутствует' Отсутствует
could not convert string

## Город, Федеральный округ.

Для получения этих двух признаков используем вспомогательный датасет из википедии.

In [33]:
df.iloc[:5, :13]

,Название авто,Рейтинг,Цена,Расположение,Год выпуска,Поколение,"Пробег, км",История пробега,Владельцев по ПТС,Состояние,Модификация,Объём двигателя,Тип двигателя
0,"Audi Q3 2.0 AMT, 2013, 166 000 км",4.8,1745000,"Самарская обл., Самара, Московское ш., 262А",2013,I (2011—2014),166000.0,11 записей в отчёте Автотеки,3,Не битый,2.0 TFSI quattro S tronic (170 л.с.),2 л,Бензин
1,"Audi Q7 3.0 AT, 2021, 29 226 км",5.0,8785500,"Москва, Волоколамское ш., 73",2021,4M рестайлинг (2019—2024),29226.0,2 записи в отчёте Автотеки,1,Не битый,45 TDI 3.0 quattro Tiptronic (249 л.с.),3 л,Дизель
2,"Audi Q8 3.0 AT, 2022, 100 км",5.0,11200000,"Москва, ул. Верхние Поля, 2",2022,I (2018—2023),100.0,3 записи в отчёте Автотеки,1,Не битый,55 TFSI 3.0 quattro Tiptronic (340 л.с.),3 л,Бензин
3,"Audi Q7 3.0 AT, 2013, 151 488 км",4.8,2099000,"Санкт-Петербург, пр-т Маршала Жукова, 78",2013,4L рестайлинг (2009—2015),151488.0,15 записей в отчёте Автотеки,2,Не битый,3.0 TDI quattro Tiptronic (245 л.с.),3 л,Дизель
4,"Audi Q8 3.0 AT, 2022, 3 261 км",5.0,10400000,"Москва, Бережковская наб., 38с2",2022,I (2018—2023),3261.0,NaN,1,Не битый,50 TDI 3.0 quattro Tiptronic (286 л.с.),3 л,Дизель


In [34]:
df.iloc[:5, 13:]

,Коробка передач,Привод,Комплектация,Тип кузова,Цвет,Руль,VIN или номер кузова,Управление климатом,ПТС,Обмен,Бренд авто,Модель авто
0,Робот,Полный,Base,Внедорожник 5-дверный,Чёрный,Левый,WAUZ*************,NaN,NaN,NaN,Audi,Q3
1,Автомат,Полный,Sport,Внедорожник 5-дверный,Чёрный,Левый,XW8Z*************,Климат-контроль многозонный,NaN,NaN,Audi,Q7
2,Автомат,Полный,Advance,Внедорожник 5-дверный,Чёрный,Левый,WAUZ*************,Климат-контроль многозонный,Электронный,Не интересует,Audi,Q8
3,Автомат,Полный,NaN,Внедорожник 5-дверный,Белый,Левый,XW8Z*************,Климат-контроль многозонный,NaN,Возможен,Audi,Q7
4,Автомат,Полный,Base,Внедорожник 5-дверный,Красный,Левый,WAUZ*************,NaN,NaN,NaN,Audi,Q8


In [35]:
url = (f'https://ru.wikipedia.org/wiki/%D0%A1%D0%BF%D0%B8%D1%81%D0%BE%D0%BA_'
       f'%D0%B3%D0%BE%D1%80%D0%BE%D0%B4%D0%BE%D0%B2_'
       f'%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8')
cities_df = pd.read_html(url)[0]

In [36]:
def city_name(city_str: str) -> str:
    '''Функция для получения города, в котором продают авто. 
       Принимает на вход строку, возвращает с название города. '''
    global cities_df
    try:
        for city in cities_df['Город']:
            if city in city_str:
                return city
    except Exception as err:
        print(city_str, err)

In [37]:
def fed_name(city_str: str) -> str:
    '''Функция для получения федерального округа, в котором продают авто. 
       Принимает на вход строку, возвращает строку с названием округа. '''
    global cities_df
    try:
        for city in cities_df['Город']:
            if city in city_str:
                region = cities_df[cities_df['Город'] ==
                                   city]['Федеральный округ'].values[0]
                return region
    except Exception as err:
        print(city_str, err)

In [38]:
df['Город'] = df['Расположение'].apply(city_name)
df['Федеральный округ'] = df['Расположение'].apply(fed_name)

## Поколение, Выпуск, кол-во лет.

In [39]:
def generation_name(gen_str: str) -> str:
    '''Функция получает наименование поколения из строки. 
       Принимает на вход строку, возвращает поколение в формате строки. '''
    try:
        if gen_str:
            pattern = r"(.+?)\s*\((\d{4}—\d{4})\)"
            match = re.search(pattern, gen_str.strip())
            if match:
                gen = match.group(1)
                return gen
            else:
                print(f'{gen_str} Подстрока не найдена')
        else:
            return np.NaN
    except Exception as err:
        print(gen_str, err)

In [40]:
def generation_years(gen_str: str) -> int:
    '''Функция получает количество лет производства из строки. 
       Принимает на вход строку, возвращает годы в формате целого числа. '''
    try:
        if gen_str:
            pattern = r"(.+?)\s*\((\d{4}—\d{4})\)"
            match = re.search(pattern, gen_str.strip())
            if match:
                years = match.group(2)
                splitted_years = years.split('—')
                years_num = abs(
                    int(splitted_years[0]) - int(splitted_years[1]))
                return years_num
            else:
                print(f'{gen_str} Подстрока не найдена')
        else:
            return np.NaN
    except Exception as err:
        print(gen_str, err)

In [41]:
df['Наименование поколения'] = df['Поколение'].apply(generation_name)
df['Выпуск, кол-во лет'] = df['Поколение'].apply(generation_years)

nan 'float' object has no attribute 'strip'
nan 'float' object has no attribute 'strip'
nan 'float' object has no attribute 'strip'
nan 'float' object has no attribute 'strip'
I (2021–2024) Подстрока не найдена
nan 'float' object has no attribute 'strip'
nan 'float' object has no attribute 'strip'
nan 'float' object has no attribute 'strip'
nan 'float' object has no attribute 'strip'
I (2021–2024) Подстрока не найдена


## История пробега, кол-во записей.

In [42]:
df = df.rename(columns={'История пробега': 'История пробега, кол-во записей'})

In [43]:
df['История пробега, кол-во записей'].value_counts(dropna=False)

История пробега, кол-во записей
NaN                             6215
Проверить в Автотеке            2437
7 записей в отчёте Автотеки     2257
6 записей в отчёте Автотеки     2142
8 записей в отчёте Автотеки     2119
                                ... 
71 запись в отчёте Автотеки        1
73 записи в отчёте Автотеки        1
77 записей в отчёте Автотеки       1
64 записи в отчёте Автотеки        1
99 записей в отчёте Автотеки       1
Name: count, Length: 77, dtype: int64

In [44]:
def history_count(count_str: str) -> int:
    '''Функция принимает на вход строку, возвращает 
       количество записей в истории пробега в формате целого числа. 
       Если записей нет, возвращает 0'''
    try:
        match = re.search(r'\d*', count_str)[0]
        if match:
            return int(match)
        else:
            return 0
    except Exception as err:
        print(count_str, err)

In [45]:
df['История пробега, кол-во записей'] = df[
    'История пробега, кол-во записей'].apply(history_count)

nan expected string or bytes-like object
nan expected string or bytes-like object
nan expected string or bytes-like object
nan expected string or bytes-like object
nan expected string or bytes-like object
nan expected string or bytes-like object
nan expected string or bytes-like object
nan expected string or bytes-like object
nan expected string or bytes-like object
nan expected string or bytes-like object
nan expected string or bytes-like object
nan expected string or bytes-like object
nan expected string or bytes-like object
nan expected string or bytes-like object
nan expected string or bytes-like object
nan expected string or bytes-like object
nan expected string or bytes-like object
nan expected string or bytes-like object
nan expected string or bytes-like object
nan expected string or bytes-like object
nan expected string or bytes-like object
nan expected string or bytes-like object
nan expected string or bytes-like object
nan expected string or bytes-like object
nan expected str

In [46]:
df[df['История пробега, кол-во записей'] == 0][:5]

,Название авто,Рейтинг,Цена,Расположение,Год выпуска,Поколение,"Пробег, км","История пробега, кол-во записей",Владельцев по ПТС,Состояние,Модификация,Объём двигателя,Тип двигателя,Коробка передач,Привод,Комплектация,Тип кузова,Цвет,Руль,VIN или номер кузова,Управление климатом,ПТС,Обмен,Бренд авто,Модель авто,Город,Федеральный округ,Наименование поколения,"Выпуск, кол-во лет"
7,"Audi Q5 2.0 AMT, 2019, 44 540 км",4.9,4800000,"Республика Башкортостан, Уфа, мкр-н Глумилино,...",2019,II (2016—2020),44540.0,0.0,1,Не битый,40 TDI 2.0 quattro S tronic (190 л.с.),2 л,Дизель,Робот,Полный,NaN,Внедорожник 5-дверный,Чёрный,Левый,WAUZ*************,NaN,Электронный,Не интересует,Audi,Q5,Уфа,Приволжский,II,4.0
13,"Audi Q7 3.0 AT, 2020, 130 000 км",5.0,6990000,"Москва, Каширское ш., 148к3",2020,4M рестайлинг (2019—2024),130000.0,0.0,1,Не битый,45 TDI 3.0 quattro Tiptronic (249 л.с.),3 л,Дизель,Автомат,Полный,NaN,Внедорожник 5-дверный,Белый,Левый,WAUZ*************,NaN,Электронный,Не интересует,Audi,Q7,Москва,Центральный,4M рестайлинг,5.0
14,"Audi Q5 Sportback 2.0 AMT, 2022, 50 км",NaN,8349900,"Московская обл., Балашиха, территория Западная...",2022,FY (2020—2024),50.0,0.0,1,Не битый,40 TDI 2.0 quattro S tronic (204 л.с.),2 л,Дизель,Робот,Полный,Base,Внедорожник 5-дверный,Чёрный,Левый,WAUZ*************,Климат-контроль многозонный,NaN,Возможен,Audi,Q5 Sportback,Балашиха,Центральный,FY,4.0
27,"Audi Q5 2.0 AMT, 2020, 116 669 км",4.9,3979900,"Московская обл., Балашиха, территория Западная...",2020,II (2016—2020),116669.0,0.0,2,Не битый,45 TFSI 2.0 quattro S tronic (249 л.с.),2 л,Бензин,Робот,Полный,Basis,Внедорожник 5-дверный,Белый,Левый,WAUZ*************,Климат-контроль однозонный,NaN,Возможен,Audi,Q5,Балашиха,Центральный,II,4.0
31,"Audi Q7 3.0 AT, 2020, 34 000 км",5.0,7650000,"Москва, Привольная ул., 2с1",2020,4M рестайлинг (2019—2024),34000.0,0.0,1,Не битый,55 TFSIe 3.0 quattro Tiptronic (381 л.с.),3 л,Гибрид,Автомат,Полный,Base,Внедорожник 5-дверный,Синий,Левый,WAUZ*************,Климат-контроль многозонный,Электронный,Не интересует,Audi,Q7,Москва,Центральный,4M рестайлинг,5.0


## Год выпуска.

In [47]:
df['Год выпуска'].value_counts(dropna=False)

Год выпуска
2023    4321
2008    2660
2012    2566
2013    2297
2011    2185
        ... 
1987       1
1995       1
2004       1
1989       1
1934       1
Name: count, Length: 106, dtype: int64

In [48]:
df['Год выпуска'] = pd.to_numeric(df['Год выпуска'], errors='coerce')

## Владельцев по ПТС.

In [49]:
df['Владельцев по ПТС'].value_counts(dropna=False)

Владельцев по ПТС
1      9247
4+     8554
3      7717
2      7221
NaN    4042
1.0     682
2.0      88
3.0      21
Name: count, dtype: int64

In [50]:
df.loc[df['Владельцев по ПТС'] == '4+', 'Владельцев по ПТС'] = 4

In [51]:
df['Владельцев по ПТС'] = pd.to_numeric(df['Владельцев по ПТС'],
                                        errors='coerce')

In [52]:
df['Владельцев по ПТС'].value_counts(dropna=False)

Владельцев по ПТС
1.0    9929
4.0    8554
3.0    7738
2.0    7309
NaN    4042
Name: count, dtype: int64

## Состояние.

In [53]:
df['Состояние'].value_counts(dropna=False)

Состояние
Не битый    37061
Битый         507
NaN             4
Name: count, dtype: int64

In [54]:
df[df['Состояние'].isna()]

,Название авто,Рейтинг,Цена,Расположение,Год выпуска,Поколение,"Пробег, км","История пробега, кол-во записей",Владельцев по ПТС,Состояние,Модификация,Объём двигателя,Тип двигателя,Коробка передач,Привод,Комплектация,Тип кузова,Цвет,Руль,VIN или номер кузова,Управление климатом,ПТС,Обмен,Бренд авто,Модель авто,Город,Федеральный округ,Наименование поколения,"Выпуск, кол-во лет"
7685,"Citroen C4 Picasso 1.6 MT, 2010, 237 886 км",4.4,799000,"Ярославская обл., Ярославль, пр-т Октября, 73А",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Citroen,C4 Picasso,Ярославль,Центральный,None,NaN
8323,"EXEED VX 2.0 AT, 2023, 80 км",4.7,4700000,"Республика Крым, Симферопольский р-н, Молодёжн...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EXEED,VX,Сим,Уральский,None,NaN
10585,"Geely GS 1.8 AMT, 2020, 45 000 км",NaN,1699000,"Республика Адыгея, Тахтамукайский р-н, Старобж...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Geely,GS,None,None,None,NaN
10840,"Geely Monjaro 2.0 AT, 2023, 25 км",4.4,3750000,"Республика Татарстан, Казань, ул. Космонавтов, 73",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Geely,Monjaro,Казань,Приволжский,None,NaN


In [55]:
df = df.drop(df[df['Состояние'].isna()].index,
             axis=0).reset_index().drop('index', axis=1)

## Модификация, Мощность двигателя, лс.

In [56]:
df['Модификация'].value_counts(dropna=False)

Модификация
1.6 AT (123 л.с.)                          574
2.0 AT (150 л.с.)                          495
1.6 AT (120 л.с.)                          465
1.4 MT (75 л.с.)                           429
1.6 MT (98 л.с.)                           421
                                          ... 
3.5 AT (258 л.с.) Lang                       1
V 300 d 2.0 4MATIC AT (237 л.с.)             1
260 2.6 AT (160 л.с.)                        1
S350d L 2.9 4MATIC 9G-Tronic (249 л.с.)      1
2.9 4WD MT (110 л.с.)                        1
Name: count, Length: 3426, dtype: int64

In [57]:
def modification_power(mod_str: str) -> int:
    '''Функция получает количество лошадиных сил из строки. 
       Принимает на вход строку, возвращает 
       лошадиные силы в формате целого числа. '''
    try:
        if mod_str:
            pattern = r"(.+?)\s*\((\d+)\s*л\.с\.\)"
            match = re.match(pattern, mod_str.strip())
            if match:
                power = match.group(2)
                return int(power)
            else:
                print(f'{mod_str} Подстрока не найдена')
        else:
            return np.NaN
    except Exception as err:
        print(mod_str, err)

In [58]:
def modification_name(mod_str: str) -> int:
    '''Функция получает название модификации из строки. 
       Принимает на вход строку, возвращает модификацию в формате строки. '''
    try:
        if mod_str:
            pattern = r"(.+?)\s*\((\d+)\s*л\.с\.\)"
            match = re.match(pattern, mod_str.strip())
            if match:
                modification = match.group(1)
                return modification
            else:
                print(f'{mod_str} Подстрока не найдена')
        else:
            return np.NaN
    except Exception as err:
        print(mod_str, err)

In [59]:
df['Мощность двигателя, лс'] = df['Модификация'].apply(modification_power)
df['Модификация'] = df['Модификация'].apply(modification_name)

1.7 CRDI MT (136 л.с) Подстрока не найдена
2.5 4WD AT (227 л.c.) Подстрока не найдена
2.5 4WD AT (227 л.c.) Подстрока не найдена
0.0 AT (109 л.с) Подстрока не найдена
0.0 AT (150 л.с) Подстрока не найдена
0.0 AT (109 л.с) Подстрока не найдена
0.0 AT (109 л.с) Подстрока не найдена
2.0 TDI DSG (150 л.с. ) Подстрока не найдена
2.0 TDI DSG (150 л.с. ) Подстрока не найдена
2.5 AT (200 л.с) Подстрока не найдена
2.4 4WD AT (265 л.с) Подстрока не найдена
1.8 TSI DSG (152 л.с) Подстрока не найдена
1.8 TSI DSG (152 л.с) Подстрока не найдена
1.8 TSI DSG (152 л.с) Подстрока не найдена
2.0 TDI MT (140 л.с) Подстрока не найдена
1.8 TSI DSG (152 л.с) Подстрока не найдена
1.8 TSI DSG (152 л.с) Подстрока не найдена
1.8 TSI DSG (152 л.с) Подстрока не найдена
1.8 TSI DSG (152 л.с) Подстрока не найдена
1.8 TSI DSG (152 л.с) Подстрока не найдена
1.8 TSI DSG (152 л.с) Подстрока не найдена
1.8 TSI DSG (152 л.с) Подстрока не найдена
1.8 TSI DSG (152 л.с) Подстрока не найдена
1.8 TSI MT (152 л.с) Подстрока не 

In [60]:
df[df['Модификация'].str.contains('(150 л.с. )|(200 л.с)|(227 л.c.)|(152 л.с)',
                                  na=False)]

,Название авто,Рейтинг,Цена,Расположение,Год выпуска,Поколение,"Пробег, км","История пробега, кол-во записей",Владельцев по ПТС,Состояние,Модификация,Объём двигателя,Тип двигателя,Коробка передач,Привод,Комплектация,Тип кузова,Цвет,Руль,VIN или номер кузова,Управление климатом,ПТС,Обмен,Бренд авто,Модель авто,Город,Федеральный округ,Наименование поколения,"Выпуск, кол-во лет","Мощность двигателя, лс"


In [61]:
df.iloc[:5, 8:]

,Владельцев по ПТС,Состояние,Модификация,Объём двигателя,Тип двигателя,Коробка передач,Привод,Комплектация,Тип кузова,Цвет,Руль,VIN или номер кузова,Управление климатом,ПТС,Обмен,Бренд авто,Модель авто,Город,Федеральный округ,Наименование поколения,"Выпуск, кол-во лет","Мощность двигателя, лс"
0,3.0,Не битый,2.0 TFSI quattro S tronic,2 л,Бензин,Робот,Полный,Base,Внедорожник 5-дверный,Чёрный,Левый,WAUZ*************,NaN,NaN,NaN,Audi,Q3,Самара,Приволжский,I,3.0,170.0
1,1.0,Не битый,45 TDI 3.0 quattro Tiptronic,3 л,Дизель,Автомат,Полный,Sport,Внедорожник 5-дверный,Чёрный,Левый,XW8Z*************,Климат-контроль многозонный,NaN,NaN,Audi,Q7,Волоколамск,Центральный,4M рестайлинг,5.0,249.0
2,1.0,Не битый,55 TFSI 3.0 quattro Tiptronic,3 л,Бензин,Автомат,Полный,Advance,Внедорожник 5-дверный,Чёрный,Левый,WAUZ*************,Климат-контроль многозонный,Электронный,Не интересует,Audi,Q8,Москва,Центральный,I,5.0,340.0
3,2.0,Не битый,3.0 TDI quattro Tiptronic,3 л,Дизель,Автомат,Полный,NaN,Внедорожник 5-дверный,Белый,Левый,XW8Z*************,Климат-контроль многозонный,NaN,Возможен,Audi,Q7,Жуков,Центральный,4L рестайлинг,6.0,245.0
4,1.0,Не битый,50 TDI 3.0 quattro Tiptronic,3 л,Дизель,Автомат,Полный,Base,Внедорожник 5-дверный,Красный,Левый,WAUZ*************,NaN,NaN,NaN,Audi,Q8,Москва,Центральный,I,5.0,286.0


## Объём двигателя.

In [62]:
df['Объём двигателя'].value_counts(dropna=False)

Объём двигателя
1.6 л       9418
2 л         8048
1.5 л       4485
1.4 л       2330
1.8 л       2016
3 л         1791
2.5 л       1771
2.4 л       1594
2.2 л        647
3.5 л        601
1.7 л        417
2.3 л        415
1.2 л        372
2.9 л        364
1.3 л        363
2.7 л        352
2.8 л        303
4.4 л        301
2.1 л        172
3.2 л        164
5 л          156
5.7 л        141
NaN          126
1 л          125
1.9 л        112
4.5 л        108
3.6 л        104
4.7 л         96
4.6 л         93
4 л           86
4.2 л         72
5.5 л         71
0.7 л         66
3.3 л         43
2.6 л         36
1.1 л         31
3.8 л         24
3.1 л         19
4.8 л         17
5.4 л         15
3.4 л         14
4.3 л         14
0.8 л         13
6.2 л         11
3.7 л         10
5.3 л          9
6 л            6
4.1 л          6
5.6 л          6
3.9 л          3
5.2 л          3
0.6 л          2
5.8 л          2
5.9 л          1
4.9 л          1
6.4 л          1
331805 л       1
Name: count, dt

In [63]:
df['Объём двигателя'] = df['Объём двигателя'].map(
    lambda x: x.replace('\xa0л', ''), na_action='ignore')
df['Объём двигателя'] = pd.to_numeric(df['Объём двигателя'], errors='coerce')

In [64]:
df = df.rename(columns={'Объём двигателя': 'Объём двигателя, л'})

## Проверка оставшихся признаков и сохранение датасета.

In [65]:
df['Тип двигателя'].value_counts(dropna=False)

Тип двигателя
Бензин     32414
Дизель      4775
Гибрид       292
Электро       59
Газ           28
Name: count, dtype: int64

In [66]:
df['Коробка передач'].value_counts(dropna=False)

Коробка передач
Автомат     16107
Механика    12859
Робот        5439
Вариатор     3163
Name: count, dtype: int64

In [67]:
df['Привод'].value_counts(dropna=False)

Привод
Передний    22550
Полный      11877
Задний       3141
Name: count, dtype: int64

In [68]:
df['Комплектация'].value_counts(dropna=False)

Комплектация
Базовая             9150
NaN                 7278
Base                1621
Comfort             1135
Premium              683
                    ... 
Heritage Edition       1
X-Dynamic HSE          1
SE L                   1
V8                     1
Luxe Кондиционер       1
Name: count, Length: 906, dtype: int64

In [69]:
df['Комплектация'] = np.where(df['Комплектация'] == 'Base', 'Базовая',
                              df['Комплектация'])

In [70]:
df['Комплектация'].value_counts(dropna=False)

Комплектация
Базовая             10771
NaN                  7278
Comfort              1135
Premium               683
Luxury                678
                    ...  
Heritage Edition        1
X-Dynamic HSE           1
SE L                    1
V8                      1
Luxe Кондиционер        1
Name: count, Length: 905, dtype: int64

In [71]:
df['Тип кузова'].value_counts(dropna=False)

Тип кузова
Внедорожник 5-дверный    14764
Седан                    12126
Хетчбек 5-дверный         4104
Лифтбек                   1450
Фургон                    1391
Минивэн                   1298
Универсал                 1033
Хетчбек 3-дверный          371
Купе                       250
Пикап                      242
Микроавтобус               242
Внедорожник 3-дверный      130
NaN                        111
Кабриолет                   56
Name: count, dtype: int64

In [72]:
df['Цвет'].value_counts(dropna=False)

Цвет
Белый         9185
Чёрный        8828
Серый         5817
Серебряный    3812
Синий         3265
Красный       1899
Коричневый    1173
Зелёный       1074
Бежевый        711
Голубой        571
Бордовый       327
Золотой        301
Фиолетовый     261
Оранжевый      162
Жёлтый         119
Пурпурный       56
Розовый          7
Name: count, dtype: int64

In [73]:
df['Руль'].value_counts(dropna=False)

Руль
Левый     36337
Правый     1231
Name: count, dtype: int64

In [74]:
df = df.drop('VIN или номер кузова', axis=1)

In [75]:
df.iloc[:5, 17:]

,Цвет,Руль,Управление климатом,ПТС,Обмен,Бренд авто,Модель авто,Город,Федеральный округ,Наименование поколения,"Выпуск, кол-во лет","Мощность двигателя, лс"
0,Чёрный,Левый,NaN,NaN,NaN,Audi,Q3,Самара,Приволжский,I,3.0,170.0
1,Чёрный,Левый,Климат-контроль многозонный,NaN,NaN,Audi,Q7,Волоколамск,Центральный,4M рестайлинг,5.0,249.0
2,Чёрный,Левый,Климат-контроль многозонный,Электронный,Не интересует,Audi,Q8,Москва,Центральный,I,5.0,340.0
3,Белый,Левый,Климат-контроль многозонный,NaN,Возможен,Audi,Q7,Жуков,Центральный,4L рестайлинг,6.0,245.0
4,Красный,Левый,NaN,NaN,NaN,Audi,Q8,Москва,Центральный,I,5.0,286.0


In [76]:
df['Управление климатом'].value_counts(dropna=False)

Управление климатом
NaN                            16679
Климат-контроль многозонный    10791
Кондиционер                     5279
Климат-контроль однозонный      3863
Управление на руле               738
Атермальное остекление           207
Кожа                               3
Цветной                            2
Тёмный                             2
Салон                              2
Шины и диски                       1
Противотуманные                    1
Name: count, dtype: int64

In [77]:
df['ПТС'].value_counts(dropna=False)

ПТС
Оригинал       18133
NaN            10572
Дубликат        5016
Электронный     3847
Name: count, dtype: int64

In [78]:
df['Обмен'].value_counts(dropna=False)

Обмен
Не интересует    21468
Возможен          9285
NaN               6815
Name: count, dtype: int64

In [79]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37568 entries, 0 to 37567
Data columns (total 29 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Название авто                    37568 non-null  object 
 1   Рейтинг                          36861 non-null  float64
 2   Цена                             37568 non-null  object 
 3   Расположение                     37568 non-null  object 
 4   Год выпуска                      37568 non-null  float64
 5   Поколение                        37568 non-null  object 
 6   Пробег, км                       33530 non-null  float64
 7   История пробега, кол-во записей  31357 non-null  float64
 8   Владельцев по ПТС                33530 non-null  float64
 9   Состояние                        37568 non-null  object 
 10  Модификация                      37524 non-null  object 
 11  Объём двигателя, л               37442 non-null  float64
 12  Тип двигателя     

In [80]:
df.iloc[:5, :13]

,Название авто,Рейтинг,Цена,Расположение,Год выпуска,Поколение,"Пробег, км","История пробега, кол-во записей",Владельцев по ПТС,Состояние,Модификация,"Объём двигателя, л",Тип двигателя
0,"Audi Q3 2.0 AMT, 2013, 166 000 км",4.8,1745000,"Самарская обл., Самара, Московское ш., 262А",2013.0,I (2011—2014),166000.0,11.0,3.0,Не битый,2.0 TFSI quattro S tronic,2.0,Бензин
1,"Audi Q7 3.0 AT, 2021, 29 226 км",5.0,8785500,"Москва, Волоколамское ш., 73",2021.0,4M рестайлинг (2019—2024),29226.0,2.0,1.0,Не битый,45 TDI 3.0 quattro Tiptronic,3.0,Дизель
2,"Audi Q8 3.0 AT, 2022, 100 км",5.0,11200000,"Москва, ул. Верхние Поля, 2",2022.0,I (2018—2023),100.0,3.0,1.0,Не битый,55 TFSI 3.0 quattro Tiptronic,3.0,Бензин
3,"Audi Q7 3.0 AT, 2013, 151 488 км",4.8,2099000,"Санкт-Петербург, пр-т Маршала Жукова, 78",2013.0,4L рестайлинг (2009—2015),151488.0,15.0,2.0,Не битый,3.0 TDI quattro Tiptronic,3.0,Дизель
4,"Audi Q8 3.0 AT, 2022, 3 261 км",5.0,10400000,"Москва, Бережковская наб., 38с2",2022.0,I (2018—2023),3261.0,NaN,1.0,Не битый,50 TDI 3.0 quattro Tiptronic,3.0,Дизель


In [81]:
df.iloc[-5:, 13:]

,Коробка передач,Привод,Комплектация,Тип кузова,Цвет,Руль,Управление климатом,ПТС,Обмен,Бренд авто,Модель авто,Город,Федеральный округ,Наименование поколения,"Выпуск, кол-во лет","Мощность двигателя, лс"
37563,Механика,Задний,Базовая,Фургон,Белый,Левый,NaN,Электронный,Возможен,ГАЗ,ГАЗель Next,Набережные Челны,Приволжский,III,11.0,120.0
37564,Механика,Задний,Базовая,Фургон,Белый,Левый,NaN,Оригинал,Не интересует,ГАЗ,ГАЗель 3302,Майкоп,Южный,I рестайлинг,7.0,140.0
37565,Механика,Задний,Базовая,Фургон,Белый,Левый,NaN,NaN,Не интересует,ГАЗ,ГАЗель 3302,Черкесск,Северо-Кавказский,II,14.0,107.0
37566,Механика,Полный,Базовая,Внедорожник 3-дверный,Зелёный,Левый,NaN,Оригинал,Не интересует,ГАЗ,69,Нижний Тагил,Уральский,I,19.0,55.0
37567,Механика,Задний,Базовая,Фургон,Зелёный,Левый,NaN,Оригинал,NaN,ГАЗ,ГАЗель 2705,Петрозаводск,Северо-Западный,I,9.0,100.0


In [82]:
df.shape

(37568, 29)

In [83]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37568 entries, 0 to 37567
Data columns (total 29 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Название авто                    37568 non-null  object 
 1   Рейтинг                          36861 non-null  float64
 2   Цена                             37568 non-null  object 
 3   Расположение                     37568 non-null  object 
 4   Год выпуска                      37568 non-null  float64
 5   Поколение                        37568 non-null  object 
 6   Пробег, км                       33530 non-null  float64
 7   История пробега, кол-во записей  31357 non-null  float64
 8   Владельцев по ПТС                33530 non-null  float64
 9   Состояние                        37568 non-null  object 
 10  Модификация                      37524 non-null  object 
 11  Объём двигателя, л               37442 non-null  float64
 12  Тип двигателя     

In [84]:
df['Цена'] = df['Цена'].astype('int32')

In [85]:
df.to_csv('parsed_data.csv', index=False)